In [1]:
import pandas as pd
import functions as f
import sqlalchemy as db
from tqdm import tqdm 

obtain a list of all the tickers in the sp500 to scrape

In [2]:
df = pd.read_csv('csv/sp500.csv')
symbols = list(df.Symbol)

now scrape all the s&p500 stocks

In [ ]:
for symbol in tqdm(symbols):
    f.scrape(symbol)

join them to create a large dataframe with closing prices of all the stocks 

In [ ]:
f.scrape.merge_frames()    

using sqlalchemy and pandas turn the dataframe into a database table

In [ ]:
engine = db.create_engine('sqlite:///closing_price.db')
f.scrape.add_to_db(closing_price, engine)

load the dataframe from the database

In [3]:
engine = db.create_engine('sqlite:///closing_price.db')
df_prices = pd.read_sql_table('price_close', con=engine)

clean up closing prices

In [4]:
df_prices.set_index('date', inplace=True)
df_prices.drop(columns=['index'], inplace=True)

load the jaccard scores

In [22]:
import pandas as pd
jaccard_df = pd.read_csv('jaccard.csv')

clean the jaccard scores

In [23]:
jaccard_df['Symbol'] = jaccard_df['0'].apply(lambda x: x.split()[0])
jaccard_df['Symbol'] = jaccard_df['Symbol'].apply(lambda x: x[2:-2])
jaccard_df['Year'] = jaccard_df['0'].apply(lambda x: x.split()[1])
jaccard_df['Year'] = jaccard_df['Year'].apply(lambda x: x[:-1])
jaccard_df.drop(columns = ['0'],inplace=True)
jaccard_df.drop(columns = ['Unnamed: 0'],inplace=True)
jaccard_df.columns = ['jaccard_score', 'symbol', 'year']
jaccard_df.set_index('symbol',inplace=True)

In [24]:
jaccard_df.head()

,jaccard_score,year
symbol,,
FB,0.031066,2019
FBHS,0.038371,2018
FBHS,0.035291,2019
FCX,0.008125,2018
FCX,0.017146,2019


create a dataframe 8 month returns 

In [8]:
df_returns = f.transform_to_august_returns(df_prices)

In [9]:
df_returns.head()

,2014,2015,2016,2017,2018,2019
MMM,0.044131,-0.051262,0.195614,0.178489,-0.170260,-0.086096
AOS,-0.066592,0.310123,0.160016,0.190526,-0.041281,0.079595
ABT,0.065735,0.102895,-0.078966,0.247375,0.037591,0.215683
ABBV,0.087763,0.039763,0.088526,0.160391,-0.053856,-0.177407
ACN,-0.001843,0.103782,0.114799,0.064142,0.061037,0.329184


create two datasets of 2019 and 2018 returns with their jaccard score

In [31]:
df_2018 = pd.DataFrame(df_returns['2018']).join(pd.DataFrame(jaccard_df[jaccard_df['year']=='2018']), how='right')
df_2019 = pd.DataFrame(df_returns['2019']).join(pd.DataFrame(jaccard_df[jaccard_df['year']=='2019']), how='right')

In [32]:
df_2018.head()

,2018,jaccard_score,year
symbol,,,
FBHS,-0.223451,0.038371,2018
FCX,-0.132018,0.008125,2018
FDX,-0.109511,0.023673,2018
FE,0.175211,0.026320,2018
FFIV,0.316036,0.046850,2018


In [33]:
df_2019.head()

,2019,jaccard_score,year
symbol,,,
FB,0.422465,0.031066,2019
FBHS,0.491948,0.035291,2019
FCX,0.105212,0.017146,2019
FE,0.163943,0.026775,2019
FIS,0.228138,0.035282,2019
